In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Imports


In [ ]:
# Import Dependencies
%matplotlib inline

# Start Python Imports
import math, time, random, datetime

# Data Manipulation
import numpy as np
import pandas as pd

# Visualization 
import matplotlib.pyplot as plt
import missingno
import seaborn as sns
plt.style.use('seaborn-whitegrid')

# Preprocessing
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, label_binarize
# Machine learning
import catboost
from sklearn.model_selection import train_test_split
from sklearn import model_selection, tree, preprocessing, metrics, linear_model
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier, Pool, cv

# Let's be rebels and ignore warnings for now
import warnings
warnings.filterwarnings('ignore')


# Read Data

In [ ]:
train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')
sample_submission = pd.read_csv('../input/house-prices-advanced-regression-techniques/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
train.shape


In [ ]:
train.info()


In [ ]:
test.head()


In [ ]:
test.shape

In [ ]:
test.info()

In [ ]:
sample_submission.head()

In [ ]:
sample_submission.info()

# Check Missing Data

In [ ]:
sns.heatmap(train.isnull(),yticklabels=False,cbar=False )


In [ ]:
train.isnull().sum()


In [ ]:
# fill With Mean if it Mumeric 
train['LotFrontage']=train['LotFrontage'].fillna(train['LotFrontage'].mean())

# fill With Mode if it Categorical 

train['BsmtCond']=train['BsmtCond'].fillna(train['BsmtCond'].mode()[0])
train['BsmtQual']=train['BsmtQual'].fillna(train['BsmtQual'].mode()[0])
train['FireplaceQu']=train['FireplaceQu'].fillna(train['FireplaceQu'].mode()[0])
train['GarageType']=train['GarageType'].fillna(train['GarageType'].mode()[0])
train['GarageFinish']=train['GarageFinish'].fillna(train['GarageFinish'].mode()[0])
train['GarageQual']=train['GarageQual'].fillna(train['GarageQual'].mode()[0])
train['GarageCond']=train['GarageCond'].fillna(train['GarageCond'].mode()[0])
train['MasVnrType']=train['MasVnrType'].fillna(train['MasVnrType'].mode()[0])
train['MasVnrArea']=train['MasVnrArea'].fillna(train['MasVnrArea'].mode()[0])
train['BsmtExposure']=train['BsmtExposure'].fillna(train['BsmtExposure'].mode()[0])
train['BsmtFinType2']=train['BsmtFinType2'].fillna(train['BsmtFinType2'].mode()[0])



# Drop Feature

train.drop(['Alley'],axis=1,inplace=True)
train.drop(['GarageYrBlt'],axis=1,inplace=True)
train.drop(['PoolQC','Fence','MiscFeature'],axis=1,inplace=True)
train.drop(['Id'],axis=1,inplace=True)
train.dropna(inplace=True)








In [ ]:
sns.heatmap(train.isnull(),yticklabels=False,cbar=False )


In [ ]:
train.shape

In [ ]:
train.info()

In [ ]:
train.head()

# Analysis Data

In [ ]:
fig = train.hist(figsize=(9,9))


In [ ]:
sns.distplot(train['SalePrice'], kde=False, bins=8)



In [ ]:
sns.lineplot(x="SaleCondition", y="SalePrice", data=train)



# Handel Categorical Data

In [ ]:
column_name_cat=['MSZoning','Street','LotShape','LandContour','Utilities','LotConfig','LandSlope','Neighborhood',
         'Condition2','BldgType','Condition1','HouseStyle','SaleType',
        'SaleCondition','ExterCond',
         'ExterQual','Foundation','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2',
        'RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','Heating','HeatingQC',
         'CentralAir',
         'Electrical','KitchenQual','Functional',
         'FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond','PavedDrive']

In [ ]:

def Hot_Encoded_Cat(Nams_list ,train_Dataset):


    for name in Nams_list:
        frames=pd.get_dummies(train_Dataset[name],drop_first=True)
        train_Dataset.drop([name],axis=1,inplace=True)
        train_Dataset = pd.concat([train_Dataset, frames], axis=1)
        

        

    return train_Dataset
        
    

# Handel Test

In [ ]:
sns.heatmap(test.isnull(),yticklabels=False,cbar=False )


In [ ]:
test.isnull().sum()


In [ ]:
# fill With Mean if it Mumeric 
test['LotFrontage']=test['LotFrontage'].fillna(test['LotFrontage'].mean())

# fill With Mode if it Categorical 
test['MSZoning']=test['MSZoning'].fillna(test['MSZoning'].mode()[0])
test['BsmtCond']=test['BsmtCond'].fillna(test['BsmtCond'].mode()[0])
test['BsmtQual']=test['BsmtQual'].fillna(test['BsmtQual'].mode()[0])
test['FireplaceQu']=test['FireplaceQu'].fillna(test['FireplaceQu'].mode()[0])
test['GarageType']=test['GarageType'].fillna(test['GarageType'].mode()[0])
test['GarageFinish']=test['GarageFinish'].fillna(test['GarageFinish'].mode()[0])
test['GarageQual']=test['GarageQual'].fillna(test['GarageQual'].mode()[0])
test['GarageCond']=test['GarageCond'].fillna(test['GarageCond'].mode()[0])
test['MasVnrType']=test['MasVnrType'].fillna(test['MasVnrType'].mode()[0])
test['MasVnrArea']=test['MasVnrArea'].fillna(test['MasVnrArea'].mode()[0])
test['BsmtExposure']=test['BsmtExposure'].fillna(test['BsmtExposure'].mode()[0])
test['BsmtFinType2']=test['BsmtFinType2'].fillna(test['BsmtFinType2'].mode()[0])
test['Utilities']=test['Utilities'].fillna(test['Utilities'].mode()[0])
test['Exterior1st']=test['Exterior1st'].fillna(test['Exterior1st'].mode()[0])
test['Exterior2nd']=test['Exterior2nd'].fillna(test['Exterior2nd'].mode()[0])
test['BsmtFinType1']=test['BsmtFinType1'].fillna(test['BsmtFinType1'].mode()[0])
test['BsmtFinSF1']=test['BsmtFinSF1'].fillna(test['BsmtFinSF1'].mean())
test['BsmtFinSF2']=test['BsmtFinSF2'].fillna(test['BsmtFinSF2'].mean())
test['BsmtUnfSF']=test['BsmtUnfSF'].fillna(test['BsmtUnfSF'].mean())
test['TotalBsmtSF']=test['TotalBsmtSF'].fillna(test['TotalBsmtSF'].mean())
test['BsmtFullBath']=test['BsmtFullBath'].fillna(test['BsmtFullBath'].mode()[0])
test['BsmtHalfBath']=test['BsmtHalfBath'].fillna(test['BsmtHalfBath'].mode()[0])
test['KitchenQual']=test['KitchenQual'].fillna(test['KitchenQual'].mode()[0])
test['Functional']=test['Functional'].fillna(test['Functional'].mode()[0])
test['GarageCars']=test['GarageCars'].fillna(test['GarageCars'].mean())
test['GarageArea']=test['GarageArea'].fillna(test['GarageArea'].mean())

# Drop Feature
test.drop(['Alley'],axis=1,inplace=True)
test.drop(['GarageYrBlt'],axis=1,inplace=True)
test.drop(['PoolQC','Fence','MiscFeature'],axis=1,inplace=True)
test.drop(['Id'],axis=1,inplace=True)







In [ ]:
sns.heatmap(test.isnull(),yticklabels=False,cbar=False )


In [ ]:
ConCat_DF =pd.concat([train,test],axis=0)


In [ ]:
ConCat_DF.head()

In [ ]:
ConCat_DF.shape


In [ ]:
Final_Result = Hot_Encoded_Cat(column_name_cat, ConCat_DF)

In [ ]:
Final_Result.shape

In [ ]:
Final_Result =Final_Result.loc[:,~Final_Result.columns.duplicated()]


In [ ]:
Final_Result

In [ ]:
Final_Result.shape

# Split Data

In [ ]:
DataFrame_Train=Final_Result.iloc[:1422,:]
DataFrame_Test=Final_Result.iloc[1422:,:]

In [ ]:
DataFrame_Test.drop(['SalePrice'],axis=1,inplace=True)


In [ ]:
X_train=DataFrame_Train.drop(['SalePrice'],axis=1)
y_train=DataFrame_Train['SalePrice']

# Build Model

Linear Regression

In [ ]:
regressor = LinearRegression()

regressor.fit(X_train, y_train)
Acc= regressor.score(X_train, y_train)

print("Accurcy For Linear regression is  :" , Acc)


In [ ]:
y_pred = regressor.predict(DataFrame_Test)

y_pred =np.array(y_pred)

# Submission

In [ ]:


submission = pd.DataFrame()
dataframe_Y=pd.DataFrame(y_pred, columns=['SalePrice']) 

submission=pd.concat([sample_submission['Id'],dataframe_Y],axis=1)

submission.to_csv('./Submit.csv',index=False)
